# SQLite for Data Scientists

#### Produced & Presented by Florents Tselai - [tselai.com](tselai.com)

# 4. Implementing Full-Text Search (FTS5)

### Virtual Tables in SQLite

A virtual table is an *object that is registered with an open SQLite database connection*. 

From the perspective of an SQL statement, the virtual table object looks like any other table or view. But behind the scenes, queries and updates on a virtual table invoke callback methods of the virtual table object instead of reading and writing on the database file.

The virtual table mechanism allows an application to **publish interfaces** that are accessible from SQL statements as if they were tables. 

SQL statements can do almost anything to a virtual table that they can do to a real table, with the following exceptions:

* One cannot create a trigger on a virtual table.
* One cannot create additional indices on a virtual table. (Virtual tables can have indices but that must be built into the virtual table implementation. Indices cannot be added separately using CREATE INDEX statements.)
* One cannot run ALTER TABLE ... ADD COLUMN commands against a virtual table.
* Individual virtual table implementations might impose additional constraints. For example, some virtual implementations might provide read-only tables. Or some virtual table implementations might allow INSERT or DELETE but not UPDATE. Or some virtual table implementations might limit the kinds of UPDATEs that can be made.

A virtual table might represent an in-memory data structures. Or it might represent a view of data on disk that is not in the SQLite format. Or the application might compute the content of the virtual table on demand.

## FTS5

In [1]:
import json
import pandas as pd
from sqlite3 import connect

DB_PATH = '../sqlite-olt.db'

In [2]:
with connect(DB_PATH) as db:
    comments_text_df = pd.read_sql("""
                    select 
                    json_extract(data, '$.objectID') as objectID,
                    json_extract(data, '$.author') as author,
                    json_extract(data, '$.comment_text') as comment_text,
                    json_extract(data, '$._tags') as tags,
                    length(json_extract(data, '$.comment_text')) as comment_text_length
                    from hn_items_raw
                    where comment_text notnull and tags notnull
                    """,db
                    )

In [3]:
comments_text_df['comment_text'][0]

'One of my proudest moments was finding a bug in SQLite where a corrupted index caused a select statement to segfault Firefox.<p>I jumped through a <i>lot</i> of hoops to get to the point where I got a backtrace that showed me the SQL statement of a corrupted places.sqlite. I then loaded SQLite on the data file, ran the statement and reproduced the segfault. One of their lead devs then got in contact with me, grabbed the data file and fixed the issue.<p>I suspect that not only did my diagnosis lead to a fix for a LOT of Firefox crashes, but it stopped a lot of frustrating crashes on things like iPhones, etc :-)<p>I may not have done the fix, but I took the time to reproduce the problem. It felt damn good :-)<p>P.S. in case anyone is interested, the bug is <a href="https://bugzilla.mozilla.org/show_bug.cgi?id=581946" rel="nofollow">https://bugzilla.mozilla.org/show_bug.cgi?id=581946</a> on Mozilla, and at SQLite it\'s at <a href="http://www.sqlite.org/src/ci/83395a3d24" rel="nofollow">h

FTS5 is an SQLite virtual table module that provides full-text search functionality to database applications. In their most elementary form, full-text search engines allow the user to efficiently search a large collection of documents for the subset that contain one or more instances of a search term. The search functionality provided to world wide web users by Google is, among other things, a full-text search engine, as it allows users to search for all documents on the web that contain, for example, the term "fts5".

To use FTS5, the user creates an FTS5 virtual table with one or more columns. For example:

CREATE VIRTUAL TABLE email USING fts5(sender, title, body);
It is an error to add types, constraints or PRIMARY KEY declarations to a CREATE VIRTUAL TABLE statement used to create an FTS5 table. Once created, an FTS5 table may be populated using INSERT, UPDATE or DELETE statements like any other table. Like any other table with no PRIMARY KEY declaration, an FTS5 table has an implicit INTEGER PRIMARY KEY field named rowid.

In [4]:
with connect(DB_PATH) as db:
    db.execute("""
    DROP TABLE IF EXISTS comments_fts;
    """)
    db.execute("""
    CREATE VIRTUAL TABLE comments_fts USING fts5(objectID, author, comment_text);
    """)
    
    db.execute("""
    DELETE FROM comments_fts;
    """)

In [5]:
with connect(DB_PATH) as db:
    comments_text_df[['objectID', 'author', 'comment_text']].to_sql('comments_fts', db, if_exists='append', index=False)

Once populated, there are three ways to execute a full-text query against the contents of an FTS5 table:

* Using a MATCH operator in the WHERE clause of a SELECT statement, or
* Using an equals ("=") operator in the WHERE clause of a SELECT statement, or
* using the table-valued function syntax.


If using the MATCH or = operators, the expression to the left of the MATCH operator is usually the **name of the FTS5 table** (the exception is when specifying a column-filter). 

We usually search on the **whole table**

In [6]:
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    select *
                    from comments_fts
                    where comments_fts MATCH 'bane'
                    """,db
                    )
search_df

,objectID,author,comment_text
0,5758957,bane,"Fun SQLite story, I had a project that needed ..."
1,4169320,bane,Sqlite is really just one of those rare softwa...
2,5760534,bane,(once again why HN is awesome)<p>Thank you tha...
3,5759117,bane,One thing to ponder about single user apps run...
4,6816146,bane,Anybody know how the defect rates in SQLite co...
...,...,...,...
706,6816146,bane,Anybody know how the defect rates in SQLite co...
707,7886806,bane,Wait until you hear about &quot;fractal storag...
708,4661159,bane,I wonder if anybody has dared guestimate how m...
709,4740250,bane,It's a fun way to find sequential tokens (word...


In [7]:
search_df.iloc[0]['comment_text']

'Fun SQLite story, I had a project that needed to do reasonably large scale data processing (gigabytes of data) on a pretty bare boned machine in a pretty bare boned environment at a customer location. I had a fairly up-to-date perl install and notepad. For the process the data needed to look at any single element in the data and find elements similar to it. I thought long and hard about various complex data structures and caching bits to disk to make portions of the problem set fit into memory, and various ways of searching the data. It was okay if the process took a couple days to finish.<p>It suddenly hit me, I wonder if the sysadmins had installed DBD::SQLite...yes they had!<p>Why mess with all that crap if I could just use SQLite. Reasonably fast indexing, expressive searching tools, I could dump the data onto disk <i>and</i> use an in memory SQLite db at the same time, caching solved. It turned weeks of agonizing work (producing fragile code) and turned it into a quick 2 week cod

We can of course search on specific columns

In [8]:
%%timeit
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    select *
                    from comments_fts
                    where comment_text MATCH 'SQLite'
                    """,db
                    )
search_df

236 ms ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    select *
                    from comments_fts
                    where comment_text LIKE '%SQLite%'
                    """,db
                    )
search_df

268 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can also order by relevance

In [10]:
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    select *
                    from comments_fts
                    where comment_text MATCH 'SQLite'
                    order by rank
                    """,db
                    )
search_df

,objectID,author,comment_text
0,5759321,techtalsky,"Just make a quick, loose, all-text-fields sche..."
1,5759321,techtalsky,"Just make a quick, loose, all-text-fields sche..."
2,5759321,techtalsky,"Just make a quick, loose, all-text-fields sche..."
3,5759321,techtalsky,"Just make a quick, loose, all-text-fields sche..."
4,5759321,techtalsky,"Just make a quick, loose, all-text-fields sche..."
...,...,...,...
74334,7180991,gibybo,That was a lot more sophisticated than I expec...
74335,7180991,gibybo,That was a lot more sophisticated than I expec...
74336,7180991,gibybo,That was a lot more sophisticated than I expec...
74337,7180991,gibybo,That was a lot more sophisticated than I expec...


Auxiliary functions can be used to retrieve extra information regarding the matched row. 

For example, an auxiliary function may be used to retrieve a copy of a column value for a matched row with all instances of the matched term surrounded by html <b></b> tags.

In [11]:
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    SELECT highlight(comments_fts, 2, '<b>', '</b>') as matches
                    FROM comments_fts
                    WHERE comment_text MATCH 'SQLite' and comment_text MATCH 'redis'
                    """,db
                    )
search_df

,matches
0,Note that this is not a database <i>server</i>...
1,&gt; <b>Redis</b> is probably the only top-use...
2,There seem to be lots of people here that are ...
3,Quite honestly i often do not understand the e...
4,"This must be the same place I work, because no..."
...,...
2049,What kind of database? An embedded key&#x2F;va...
2050,I think this approach manages to combine all t...
2051,Probably a good idea to have antirez feedback ...
2052,"&#62;&#62; Really, unless you run it on a ""ser..."


In [12]:
search_df['matches'][1]

'&gt; <b>Redis</b> is probably the only top-used database system developed mostly by a single individual currently<p>isn&#x27;t <b>SQLite</b> pretty much just Richard Hipp?'

### Searching for strings

In [13]:
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    SELECT highlight(comments_fts, 2, '<b>', '</b>') as matches
                    FROM comments_fts
                    WHERE comment_text MATCH '"database system"'
                    """,db
                    )
search_df

,matches
0,&gt; Redis is probably the only top-used <b>da...
1,&gt; Redis is probably the only top-used <b>da...
2,&gt; Redis is probably the only top-used <b>da...
3,&gt; Redis is probably the only top-used <b>da...
4,&gt; Redis is probably the only top-used <b>da...
...,...
74,&gt; Redis is probably the only top-used <b>da...
75,&gt; Redis is probably the only top-used <b>da...
76,&gt; Redis is probably the only top-used <b>da...
77,&gt; Redis is probably the only top-used <b>da...


In [14]:
search_df.matches[0]

'&gt; Redis is probably the only top-used <b>database system</b> developed mostly by a single individual currently<p>isn&#x27;t SQLite pretty much just Richard Hipp?'

### Searching for phrases

In [15]:
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    SELECT highlight(comments_fts, 2, '<***>', '</***>') as matches
                    FROM comments_fts
                    WHERE comment_text MATCH 'Redis + sqlite'
                    """,db
                    )
search_df

,matches
0,They <i>think</i> differently.<p>Take a look a...
1,They <i>think</i> differently.<p>Take a look a...
2,They <i>think</i> differently.<p>Take a look a...
3,They <i>think</i> differently.<p>Take a look a...
4,They <i>think</i> differently.<p>Take a look a...
...,...
74,They <i>think</i> differently.<p>Take a look a...
75,They <i>think</i> differently.<p>Take a look a...
76,They <i>think</i> differently.<p>Take a look a...
77,They <i>think</i> differently.<p>Take a look a...


In [16]:
search_df['matches'][0]

'They <i>think</i> differently.<p>Take a look at Smalltalk at Xerox PARC, AI lab and Scheme language at MIT, Plan9 at Bell Labs, Erlang at Ericsson, Go at Google, FreeBSD and OpenBSD projects, etc.<p>For an individual achievements - Linux 0.9, viaweb, arc, clisp, nginx, <***>redis, sqlite</***>, git, vi, emacs, etc.<p>Each particular story could be enough for learning almost everything about software engineering.<p>Erlang is a story of applied principles, methodology and design decisions - real-world functional programming success.<p>Plan9 is an example of what system programming really are, that less is really more and good is really enough.) Take a lot at src/lib<p>Arc is a prove of concept that lies in foundation of the "On Lisp" book - that you could evolve a language together with your software in a bottom-up process. And he did.<p>The story of Scheme language you could read in MIT AI memos, where, actually you could find all you need to know about  programming.<p>Go is an example

### Prefix queries

In [17]:
with connect(DB_PATH) as db:
    search_df = pd.read_sql("""
                    SELECT highlight(comments_fts, 2, '<***>', '</***>') as matches
                    FROM comments_fts
                    WHERE comment_text MATCH 'stats*'
                    """,db
                    )
search_df

,matches
0,"EDITED ONE LAST TIME I PROMISE, I REALLY DO, ..."
1,A few months ago I was making plans for what i...
2,File based locking. If you want to write somet...
3,Good job shipping! This was the kind of app I ...
4,"EDITED ONE LAST TIME I PROMISE, I REALLY DO, ..."
...,...
311,Good job shipping! This was the kind of app I ...
312,"EDITED ONE LAST TIME I PROMISE, I REALLY DO, ..."
313,A few months ago I was making plans for what i...
314,File based locking. If you want to write somet...


In [18]:
search_df['matches'][0]

"EDITED ONE LAST TIME  I PROMISE, I REALLY DO, THIS IS THE LAST TIME  TO ADD:<p>I think the reason that OP's join-based queries are slow is that there are no indexes over his junction table's foreign keys:<p><pre><code>    CREATE TABLE movies_people (\n      movie_id INTEGER REFERENCES movie,\n      person_id INTEGER REFERENCES person\n    );\n</code></pre>\nThus when he wants the movies associated with person 160, the database must examine the entire junction:<p><pre><code>    EXPLAIN ANALYZE SELECT * FROM movies_for_people_junction WHERE person_id = 160;\n\n    Hash Join  (cost=282.37..10401.08 rows=97 width=33) (actual time=7.440..64.843 rows=9 loops=1)\n      Hash Cond: (movie_person.movie_id = movie.id)\n      -&#62;  Seq Scan on movie_person  (cost=0.00..10117.01 rows=97 width=8) (actual time=2.540..59.933 rows=9 loops=1)\n            Filter: (person_id = 160)\n      -&#62;  Hash  (cost=233.83..233.83 rows=3883 width=29) (actual time=4.884..4.884 rows=3883 loops=1)\n            B